In [1]:
import pandas as pd
import numpy as np
import os
import sys
import matplotlib.pyplot as plt
import seaborn as sns
import json

In [2]:
from pyspark.sql import SparkSession
from pyspark import SparkContext
import  pyspark.sql.functions as F
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.sql.types import ArrayType, IntegerType, StructType, StructField, StringType
from pyspark.sql import Window

In [3]:
transactions_data_path = "../data/transactions.csv"
catalogue_path = "../data/catalogue.json"
test_path = "../data/test_users.json"
ratings_path = "../data/ratings.csv"
bookmarks_path = "../data/bookmarks.csv"

In [4]:
spark = SparkSession.builder \
            .appName("geneRating") \
            .getOrCreate()

### Bookmarks exploration

In [5]:
bm_df = spark.read.csv(bookmarks_path, header=True)

In [6]:
bm_df = bm_df.withColumn("bookmark", F.lit(1))

### Transactions exploration

Загружаем данные по транзакциям

In [7]:
tr_df = spark.read.csv(transactions_data_path, header=True)

### Загружаем каталог

In [8]:
cat_df = pd.read_json(catalogue_path, orient='index').reset_index()
cat_df = cat_df.rename(columns={'index': 'element_uid'})

In [9]:
catalogue = spark.createDataFrame(cat_df)

### Data preprocessing

Добавляем к транзакциям информацию о фильме и выбрасываем транзакции по фильмам, по которым нет информации.

In [ ]:
tr_df.count()

In [ ]:
tr_df.show(5)

Будем использовать только информацию о типе фильма и о его длительности

In [10]:
catalogue = catalogue.select(['element_uid', 'type', 'duration'])

In [11]:
df = tr_df.join(catalogue, on='element_uid', how='inner')

Видим, что транзакций по неизвестным фильмам нет

In [12]:
df = df.withColumn("watched_time", F.col("watched_time")/60)

Заметим, что 1% представленного каталога имеет длительность фильма = 0

In [ ]:
catalogue.where(catalogue['duration'] == 0).count() / catalogue.count() * 100

Заменим эту длительность на 75%й квантиль времени просмотра этих фильмов

Так как неизвестно, сколько частей в многосерийном фильме и сериале, для каждого такого фильма и сериала посчитаем 75-квантиль и возьмем его за длительность фильма/сериала. Предварительно переведем длительность просмотра в минуты.

In [13]:
quantiles = df.filter((df['type'] == "multipart_movie") | (df['type'] == "series") | (df['duration'] == 0))\
    .groupBy('element_uid').agg(F.expr("percentile_approx(watched_time, 0.75)").alias('true_duration'))

In [14]:
df = df.join(quantiles, on="element_uid", how="left")

In [15]:
df = df.withColumn("true_duration", F.coalesce("true_duration", "duration"))

Добавим колонку с информацией о том, какую часть фильма посмотрел пользователь, предварительно удалив записи, по которым неизвестна длительность просмотра (около 100к)

In [16]:
df = df.where(df['watched_time'].isNotNull())\
    .withColumn("watch_part", F.col('watched_time') / F.col('true_duration'))

Посмотрим, какую часть всех фильмов составляют фильмы с watch_part > 1

In [ ]:
multiwatch_films = df.where((df['type'] == 'movie') & (df['watch_part'] > 1))
films = df.where(df['type'] == 'movie')

print(f"Multiwatch films are {multiwatch_films.count()/films.count() * 100} perc")

Считаем, что если длительность просмотра > 1,то пользователь просмотрел фильм/сериал несколько раз. Пока не будем это учитывать и заменим зачение на 1

In [17]:
# multiwatch strategy
df = df.withColumn("true_watch_part", F.when(df['watch_part'] > 12.0, 3.0)\
                                       .when(df['watch_part'] > 1.0, 1.0 + (df['watch_part'] / 6.0))
                                       .otherwise(F.col("watch_part")))

### Cut non popular films

In [18]:
film_watches = df.groupBy('element_uid').count()
film_watches = film_watches.withColumnRenamed('count', "watch_num")

In [19]:
film_watches = film_watches.withColumn("enough_films", F.when(film_watches['watch_num']>=50, 1)\
                                                          .otherwise(0))

In [20]:
df = df.join(film_watches.select(['element_uid', 'enough_films']), on='element_uid', how='left')

In [21]:
df = df.filter(df['enough_films'] == 1)

In [23]:
df.show(5)

+-----------+--------+----------------+------------------+------------------+-----------+-------------------+-----+--------+-------------+------------------+------------------+------------+
|element_uid|user_uid|consumption_mode|                ts|      watched_time|device_type|device_manufacturer| type|duration|true_duration|        watch_part|   true_watch_part|enough_films|
+-----------+--------+----------------+------------------+------------------+-----------+-------------------+-----+--------+-------------+------------------+------------------+------------+
|       1436|   84071|               S| 44264014.90908134|             12.75|          0|                 50|movie|      10|         10.0|             1.275|            1.2125|           1|
|       1436|  351732|               S|44262414.033952884|11.233333333333333|          0|                 50|movie|      10|         10.0|1.1233333333333333|1.1872222222222222|           1|
|       1436|  178530|               S| 44164657.5

### Ratings

In [22]:
ratings_df = spark.read.csv(ratings_path, header=True)

In [28]:
ratings_df.show(5)

+--------+-----------+------+------------------+
|user_uid|element_uid|rating|                ts|
+--------+-----------+------+------------------+
|  571252|       1364|    10| 44305174.26309871|
|   63140|       3037|    10|44305139.282818206|
|  443817|       4363|     8| 44305136.20584908|
|  359870|       1364|    10| 44305063.00637321|
|  359870|       3578|     9| 44305060.73913281|
+--------+-----------+------+------------------+
only showing top 5 rows



In [23]:
df = df.join(ratings_df.select(['user_uid', 'element_uid', 'rating']),
             on=['user_uid', 'element_uid'], how='left')\
        .join(bm_df.select(['user_uid', 'element_uid', 'bookmark']),
             on=['user_uid', 'element_uid'], how='left')

In [24]:
df = df.fillna(0, subset=["bookmark"])

In [25]:
df = df.select([
    'user_uid',
    'element_uid',
    'consumption_mode',
    'ts',
    'type',
    'true_duration',
    'true_watch_part',
    'rating',
    'bookmark'
])

### StepDan evristic

In [32]:
df = df.withColumn('rating', F.col('rating') + 1)

In [33]:
user_rating_count = df.filter(df['rating'].isNotNull())\
                        .groupBy('user_uid').agg(F.count('rating').alias("rate_count"),
                                                 F.mean('rating').alias("rate_mean"))

In [34]:
user_rating_count.show(5)

+--------+----------+-----------------+
|user_uid|rate_count|        rate_mean|
+--------+----------+-----------------+
|  190265|        17|8.705882352941176|
|  214945|         8|            9.625|
|  552277|         3|9.333333333333334|
|   59652|        11|9.272727272727273|
|   70962|         2|             10.0|
+--------+----------+-----------------+
only showing top 5 rows



In [35]:
tmp_df = df.join(user_rating_count, on='user_uid', how='left')

In [36]:
user_implicit = df.groupBy('user_uid').agg(F.mean('true_watch_part').alias('mean_implicit'))

In [37]:
tmp_df = tmp_df.join(user_implicit, on='user_uid', how='left')

In [38]:
tmp_df = tmp_df.withColumn('rate', F.when((tmp_df['rate_count'] >= 5) & (tmp_df['rating'].isNotNull()),
                                         F.col('rating')/F.col('rate_mean') * F.col('mean_implicit'))\
                                   .otherwise(F.col('true_watch_part'))
                                   
                          )

In [ ]:
# cut off
#tmp_df = tmp_df.withColumn('rate', F.when(tmp_df['rate'] > 1, 1).otherwise(F.col('rate')))

In [ ]:
tmp_df.where(tmp_df['rate'].isNull()).show()

In [ ]:
tmp_df.describe('rate').show()

# parquet write

In [26]:
#запись в файл
default_cols = ['element_uid', 'user_uid', 'consumption_mode', 'ts', 'device_type', 'device_manufacturer', 'type',
              'true_duration', 'true_watch_part']

def write_to_prq(df, default_cols, path="../data/ratings_prq"):
    df.select(default_cols)\
        .repartition(100)\
        .write.parquet(path)

In [27]:
!rm -rdf "../data/StepDan_ratings_prq"

In [30]:
#запись в файл
need_cols = ['element_uid', 'user_uid', 'ts', "true_watch_part"]

write_to_prq(df, need_cols, "../data/base_rating_prq")

In [31]:
spark.stop()

## Multiwatch

In [ ]:
multiwatch_films.describe('watch_part').show()

In [ ]:
watch_times = multiwatch_films\
                .withColumn("watch_part_int", multiwatch_films["watch_part"].cast(IntegerType()))\
              .select('watch_part_int')


In [ ]:
watch_times.groupBy('watch_part_int').agg(F.count('watch_part_int'))\
            .orderBy('count(watch_part_int)', ascending=False)\
            .show()